# From SQL to pandas challenge 7

In [ ]:
# import libraries
import pandas as pd

# load data
# This code is made to load our data stored on Google Drive
def gd_path(file_id):
    """Generate a shareable link from Google Drive file id."""
    return f"https://drive.google.com/uc?export=download&id={file_id}"

# Google Drive file ids
files_id = {
    "titles": "1PLdn50N9GRa53ZbuVWo0l47F_IXdvlEm",
    "sales": "1fzFc9rwYmVIPaGOFmhLVxCi3kg19vNU2", 
    "employee": "1h9mUjsVqpP74b1w0x7KOw37n_n9Ulkt5", 
    "authors": "1fEF89Nhe61EebAljKlwFwfEuokK0o6aJ"
}

# Read data from Google Drive
authors = pd.read_csv(gd_path(files_id["authors"]), sep=";")
titles = pd.read_csv(gd_path(files_id["titles"]), sep=";")
sales = pd.read_csv(gd_path(files_id["sales"]), sep=";")
employee = pd.read_csv(gd_path(files_id["employee"]), sep=";")

## 1. Find the total amount of authors for each state

In [ ]:
authors.groupby(by='state')['au_id'].count()


state
CA    15
IN     1
KS     1
MD     1
MI     1
OR     1
TN     1
UT     2
Name: au_id, dtype: int64

### Hint:

In SQL the syntax is:

```sql
SELECT COUNT(au_id) , state
FROM authors
group by state;
```

## 2. Find the total amount of authors by each state and order them in descending order

In [ ]:
(
authors.groupby(by='state')
        ['au_id']
        .count()
        .sort_values(ascending=False)
)

state
CA    15
UT     2
IN     1
KS     1
MD     1
MI     1
OR     1
TN     1
Name: au_id, dtype: int64

### Hint:

In SQL the syntax is:

```sql
SELECT COUNT(au_id) Total,state
FROM authors
GROUP BY state
ORDER BY (total) DESC;
```

## 3. What's the price of the most expensive title from each publisher?

In [ ]:
titles.groupby(by = 'pub_id')['price'].max()

pub_id
736     19.99
877     21.59
1389    22.95
Name: price, dtype: float64

### Hint:

In SQL the syntax is:

```sql
SELECT MAX(price), pub_id
FROM titles
GROUP BY pub_id;
```

## 4. Find out the top 3 stores with the most sales

In [ ]:
(
sales.groupby(by='stor_id')
      ['qty']
      .sum()
      .sort_values(ascending = False)
      .head(n = 3)
)

stor_id
7131    130
7066    125
7067     90
Name: qty, dtype: int64

### Hint:

In SQL the syntax is:

```sql
SELECT stor_id, SUM(qty)
FROM sales
GROUP BY stor_id
ORDER BY SUM(qty) DESC
LIMIT 3;
```

## 5. Find the average job level for each job_id from the employees table.
    Order the jobs in ascending order by its average job level.

In [ ]:
(
employee.groupby(by='job_id')
        ['job_lvl']
        .mean()
        .sort_values(ascending=False)
)

job_id
4     227.000000
2     215.000000
3     200.000000
5     187.285714
6     176.000000
7     142.500000
8     125.250000
9     123.250000
11    121.750000
10    101.750000
14     74.666667
13     66.333333
12     55.666667
Name: job_lvl, dtype: float64

### Hint:

In SQL the syntax is:

```sql
SELECT AVG(job_lvl), job_id
FROM employee
GROUP BY job_id
ORDER BY AVG(job_lvl);
```

## 6. For each type (business, psychology…), find out how many books each publisher has

In [ ]:
titles.groupby(by=['type', 'pub_id'])['title_id'].count()

type          pub_id
UNDECIDED     877       1
business      736       1
              1389      3
mod_cook      877       2
popular_comp  1389      3
psychology    736       4
              877       1
trad_cook     877       3
Name: title_id, dtype: int64

### Hint:

In SQL the syntax is:

```sql
SELECT pub_id, type, COUNT(title_id)
FROM titles
GROUP BY pub_id , type;
```

## 7. Add the average price of each publisher - book type combination from your previous query

In [ ]:
(
titles.groupby(by=['type', 'pub_id'])
      .aggregate({'title_id': 'count',
                  'price': 'mean'})
)

title_id      price
type         pub_id                     
UNDECIDED    877            1   0.000000
business     736            1   2.990000
             1389           3  17.310000
mod_cook     877            2  11.490000
popular_comp 1389           3  14.316667
psychology   736            4  11.482500
             877            1  21.590000
trad_cook    877            3  15.963333

### Hint:

In SQL the syntax is:

```sql
SELECT  AVG(price), COUNT(*), pub_id,type
FROM titles
GROUP BY pub_id, type;
```

## 8. From your previous query, keep only the combinations of publisher - book type with an average price higher than 12

In [ ]:
# Create a new dataframe to make it easier to work with
pub_type = (titles.groupby(by=['type', 'pub_id'])
                  .aggregate({'title_id': 'count',
                              'price': 'mean'}))

pub_type.loc[pub_type['price']>12]

,,title_id,price
type,pub_id,,
business,1389,3,17.310000
popular_comp,1389,3,14.316667
psychology,877,1,21.590000
trad_cook,877,3,15.963333


### Hint:

In SQL the syntax is:

```sql
SELECT pub_id,type, COUNT(*)
FROM titles
GROUP BY pub_id, type
HAVING AVG(price) > 12;
```

## 9. Order the results of your previous query by these two criteria:
1. Count of books, descendingly
2. Average price, descendingly

In [ ]:
(
pub_type.loc[pub_type['price']>12]
        .sort_values(by=['title_id', 'price'], ascending = False)
)

,,title_id,price
type,pub_id,,
business,1389,3,17.310000
trad_cook,877,3,15.963333
popular_comp,1389,3,14.316667
psychology,877,1,21.590000


### Hint:

In SQL the syntax is:

```sql
SELECT  pub_id, type, COUNT(*), AVG(price)
FROM titles
GROUP BY pub_id, type
HAVING AVG(price) > 12
ORDER BY COUNT(*) DESC, AVG(price) DESC;
```

## 10. Some authors have a contract, while others don't - it's indicated in the "contract" column of the authors table.
     
    Select all the states and cities where there are 2 or more contracts overall

In [ ]:
contracts_state_city = (authors.groupby(by=['state', 'city'])
                               .aggregate({'contract': 'sum'})
)

contracts_state_city.loc[contracts_state_city['contract'] >= 2]

contract
state city                    
CA    Berkeley               2
      Oakland                4
      Palo Alto              2
UT    Salt Lake City         2

### Hint:

In SQL the syntax is:

```sql
SELECT SUM(contract),city,state
FROM authors
GROUP BY state,city
HAVING SUM(contract)>1;
```